In [4]:
import os, glob
import numpy as np
import cv2
import matplotlib.pyplot as plt

from ss_baselines.common.utils import plot_top_down_map

# ===== 你只要改这里 =====
ROOT_A = "/home/Disk/yyz/sound-spaces/vis/debug_npz"         # A 的根目录（里面有很多 scene 子文件夹）
ROOT_OTHERS = [
    "/home/Disk/yyz/sound-spaces/vis/debug_npz_ours",
    # "/home/Disk/yyz/sound-spaces/vis/debug_npz_iros",
    # "/home/Disk/yyz/sound-spaces/vis/debug_npz_ral",
    # "/home/Disk/yyz/sound-spaces/vis/debug_npz_ral",
    # "/home/Disk/yyz/sound-spaces/vis/debug_npz_avnav",
    # "/home/Disk/yyz/sound-spaces/vis/debug_npz_savi",
    # "/home/Disk/yyz/sound-spaces/vis/debug_npz_enmus",
]
OUT_DIR = "/home/Disk/yyz/sound-spaces/topdown_pngs"
DATASET_NAME = "mp3d"  # 你之前用 mp3d

os.makedirs(OUT_DIR, exist_ok=True)


In [5]:
def sorted_npz_files(d):
    files = glob.glob(os.path.join(d, "*.npz"))
    if len(files) == 0:
        return []
    # 适配你这种 0.0.npz, 1.0.npz...
    return sorted(files, key=lambda p: float(os.path.basename(p).replace(".npz","")))

def npz_to_info_topdown(d):
    xy = d["agent_map_coord"]
    return {
        "top_down_map": {
            "map": d["map"],
            "fog_of_war_mask": d["fog_of_war_mask"],
            "agent_map_coord": (int(xy[0]), int(xy[1])),   # (row,col)
            "agent_angle": float(d["agent_angle"]),
        }
    }

def load_A_world_and_rc(filesA):
    world = []
    rc = []
    for f in filesA:
        d = np.load(f)
        world.append(d["agent_pos"].astype(np.float32))          # (x,z)
        rc.append(d["agent_map_coord"].astype(np.float32))       # (row,col)
    return np.stack(world), np.stack(rc)

def load_world_only(files, key_candidates=("agent_pos", "agent_world_pos")):
    world = []
    for f in files:
        d = np.load(f)
        k = None
        for cand in key_candidates:
            if cand in d:
                k = cand
                break
        if k is None:
            raise KeyError(f"{os.path.basename(f)} has no agent_pos/agent_world_pos")
        world.append(d[k].astype(np.float32))
    return np.stack(world)

def world_to_grid_rc(world_xz, bounds, H, W, flip_z=False):
    (minx, _, minz), (maxx, _, maxz) = bounds
    x = world_xz[:, 0]
    z = world_xz[:, 1]
    c = (x - minx) / (maxx - minx + 1e-9) * (W - 1)
    if flip_z:
        r = (maxz - z) / (maxz - minz + 1e-9) * (H - 1)
    else:
        r = (z - minz) / (maxz - minz + 1e-9) * (H - 1)
    return np.stack([r, c], axis=1)  # (row,col)

def clip_rc(rc, H, W):
    r = np.clip(np.round(rc[:, 0]).astype(np.int32), 0, H - 1)
    c = np.clip(np.round(rc[:, 1]).astype(np.int32), 0, W - 1)
    return np.stack([r, c], axis=1)

def pick_best_flip(worldA, rcA, bounds, H, W):
    pred0 = world_to_grid_rc(worldA, bounds, H, W, flip_z=False)
    pred1 = world_to_grid_rc(worldA, bounds, H, W, flip_z=True)
    e0 = np.mean(np.linalg.norm(pred0 - rcA, axis=1))
    e1 = np.mean(np.linalg.norm(pred1 - rcA, axis=1))
    return (e1 < e0)

def draw_square(img, center_xy, size, color, thickness=-1):
    x, y = center_xy
    pt1 = (int(x - size), int(y - size))
    pt2 = (int(x + size), int(y + size))
    cv2.rectangle(img, pt1, pt2, color, thickness)


In [6]:
# 轨迹颜色（BGR）
COLORS = [
    (0, 180, 0),     # A：深绿
    (0, 0, 255),   # other1：黄
    (0, 255, 255),   # other1：黄
    (255, 255, 0),   # other2：青
    (255, 0, 255),   # other3：紫
    (0, 128, 255),   # other4：橙
    (255, 0, 0),     # other5：蓝
]

# A 根目录下的所有“场景子文件夹”
scene_dirs_A = sorted([p for p in glob.glob(os.path.join(ROOT_A, "*")) if os.path.isdir(p)])
print("Found scenes in A:", len(scene_dirs_A))

saved = 0
skipped = 0

for scene_dir_A in scene_dirs_A:
    scene_name = os.path.basename(scene_dir_A)
    filesA = sorted_npz_files(scene_dir_A)
    if len(filesA) == 0:
        skipped += 1
        continue

    # === 读 A 第一帧拿元信息 ===
    d0 = np.load(filesA[0])
    if not all(k in d0 for k in ["bounds", "map_shape", "map", "fog_of_war_mask", "agent_map_coord", "agent_angle", "agent_pos"]):
        print(f"[Skip] {scene_name}: missing required keys in A")
        skipped += 1
        continue

    bounds = d0["bounds"].astype(np.float32)
    H, W = map(int, d0["map_shape"])

    # === flip_z 自动选择（用 A 的 world+rc 做一致性判断）===
    worldA, rcA = load_A_world_and_rc(filesA)
    flip_z = pick_best_flip(worldA, rcA, bounds, H, W)

    # === 官方底图 ===
    info0 = npz_to_info_topdown(d0)
    img_rgb = plot_top_down_map(info0, dataset=DATASET_NAME)  # RGB
    img_bgr = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)

    # === 画 A 轨迹（用 A 的 rc 更稳）===
    pathA_rc = clip_rc(rcA, H, W)
    colorA = COLORS[0]
    for i in range(1, len(pathA_rc)):
        r1, c1 = pathA_rc[i-1]
        r2, c2 = pathA_rc[i]
        cv2.line(img_bgr, (c1, r1), (c2, r2), colorA, 4)

    # A 终点方块（红）
    r_end, c_end = pathA_rc[-1]
    draw_square(img_bgr, (c_end, r_end), size=6, color=(0, 0, 255), thickness=-1)

    # === 画其它方法（world-only）===
    for j, root_other in enumerate(ROOT_OTHERS, start=1):
        scene_dir_other = os.path.join(root_other, scene_name)
        filesO = sorted_npz_files(scene_dir_other)
        if len(filesO) == 0:
            continue

        try:
            worldO = load_world_only(filesO)
        except Exception as e:
            print(f"[Warn] {scene_name} in {os.path.basename(root_other)}: {e}")
            continue

        rcO = world_to_grid_rc(worldO, bounds, H, W, flip_z=flip_z)
        pathO_rc = clip_rc(rcO, H, W)

        color = COLORS[j % len(COLORS)]
        for i in range(1, len(pathO_rc)):
            r1, c1 = pathO_rc[i-1]
            r2, c2 = pathO_rc[i]
            cv2.line(img_bgr, (c1, r1), (c2, r2), color, 4)

        # 其它方法终点：小方块（对应颜色）
        r_end, c_end = pathO_rc[-1]
        draw_square(img_bgr, (c_end, r_end), size=5, color=color, thickness=-1)

    # === 保存 ===
    out_path = os.path.join(OUT_DIR, f"{scene_name}.png")
    cv2.imwrite(out_path, img_bgr)
    saved += 1

print("Saved:", saved, "Skipped:", skipped, "Output dir:", OUT_DIR)


Found scenes in A: 1000
Saved: 1000 Skipped: 0 Output dir: /home/Disk/yyz/sound-spaces/topdown_pngs


In [1]:
import os, glob
import numpy as np
import cv2
import matplotlib.pyplot as plt

from ss_baselines.common.utils import plot_top_down_map

# ===== 你只要改这里 =====
ROOT_A = "/home/Disk/yyz/sound-spaces/vis/debug_npz"         # A: shortest path + map
ROOT_OTHERS = [
    "/home/Disk/yyz/sound-spaces/vis/debug_npz_ours",
    "/home/Disk/yyz/sound-spaces/vis/debug_npz_iros",
    "/home/Disk/yyz/sound-spaces/vis/debug_npz_ral",
    "/home/Disk/yyz/sound-spaces/vis/debug_npz_avnav",
    "/home/Disk/yyz/sound-spaces/vis/debug_npz_savi",
    "/home/Disk/yyz/sound-spaces/vis/debug_npz_enmus",
]
OUT_DIR = "/home/Disk/yyz/sound-spaces/vis/topdown_pngs_sep"
DATASET_NAME = "mp3d"

# 只处理这些场景子文件夹名（例子）
SCENE_LIST = [
    "q9vSo1VnCiC_3017",
    "5ZKStnWn8Zo_2957",
    "5ZKStnWn8Zo_4701",
    "5ZKStnWn8Zo_4791",
    "5ZKStnWn8Zo_4806",
    "5ZKStnWn8Zo_5076",
    "5ZKStnWn8Zo_6412",
    "5ZKStnWn8Zo_6544",
    "5ZKStnWn8Zo_6644",
    "5ZKStnWn8Zo_6794",
    "5ZKStnWn8Zo_6836",
    "5ZKStnWn8Zo_6841",
    "5ZKStnWn8Zo_7233",
    "5ZKStnWn8Zo_4415",
    "5ZKStnWn8Zo_6164",
    "fzynW3qQPVF_1394",
    "fzynW3qQPVF_3935",
    "fzynW3qQPVF_11572",
    "fzynW3qQPVF_13292",
    "fzynW3qQPVF_20453",
    "fzynW3qQPVF_31143",
    "fzynW3qQPVF_31850",
    "wc2JMjhGNzB_2813",
]

os.makedirs(OUT_DIR, exist_ok=True)

def sorted_npz_files(d):
    files = glob.glob(os.path.join(d, "*.npz"))
    if len(files) == 0:
        return []
    return sorted(files, key=lambda p: float(os.path.basename(p).replace(".npz","")))

def npz_to_info_topdown(d):
    xy = d["agent_map_coord"]
    return {
        "top_down_map": {
            "map": d["map"],
            "fog_of_war_mask": d["fog_of_war_mask"],
            "agent_map_coord": (int(xy[0]), int(xy[1])),
            "agent_angle": float(d["agent_angle"]),
        }
    }

def load_A_world_and_rc(filesA):
    world, rc = [], []
    for f in filesA:
        d = np.load(f)
        world.append(d["agent_pos"].astype(np.float32))
        rc.append(d["agent_map_coord"].astype(np.float32))
    return np.stack(world), np.stack(rc)

def load_world_only(files, key_candidates=("agent_pos", "agent_world_pos")):
    world = []
    for f in files:
        d = np.load(f)
        k = None
        for cand in key_candidates:
            if cand in d:
                k = cand
                break
        if k is None:
            raise KeyError(f"{os.path.basename(f)} has no agent_pos/agent_world_pos")
        world.append(d[k].astype(np.float32))
    return np.stack(world)

def world_to_grid_rc(world_xz, bounds, H, W, flip_z=False):
    (minx, _, minz), (maxx, _, maxz) = bounds
    x = world_xz[:, 0]
    z = world_xz[:, 1]
    c = (x - minx) / (maxx - minx + 1e-9) * (W - 1)
    if flip_z:
        r = (maxz - z) / (maxz - minz + 1e-9) * (H - 1)
    else:
        r = (z - minz) / (maxz - minz + 1e-9) * (H - 1)
    return np.stack([r, c], axis=1)

def clip_rc(rc, H, W):
    r = np.clip(np.round(rc[:, 0]).astype(np.int32), 0, H - 1)
    c = np.clip(np.round(rc[:, 1]).astype(np.int32), 0, W - 1)
    return np.stack([r, c], axis=1)

def pick_best_flip(worldA, rcA, bounds, H, W):
    pred0 = world_to_grid_rc(worldA, bounds, H, W, flip_z=False)
    pred1 = world_to_grid_rc(worldA, bounds, H, W, flip_z=True)
    e0 = np.mean(np.linalg.norm(pred0 - rcA, axis=1))
    e1 = np.mean(np.linalg.norm(pred1 - rcA, axis=1))
    return (e1 < e0)

def draw_square(img, center_xy, size, color, thickness=-1):
    x, y = center_xy
    pt1 = (int(x - size), int(y - size))
    pt2 = (int(x + size), int(y + size))
    cv2.rectangle(img, pt1, pt2, color, thickness)

# ===== 颜色（BGR）=====
COLOR_SHORTEST = (0, 180, 0)   # 绿色：shortest path
COLOR_METHOD   = (255, 0, 0)   # 蓝色：method path (BGR: blue)
COLOR_END      = (0, 0, 255)   # 红色终点方块（可选）

def render_one(scene_name, method_root=None, method_tag="shortest"):
    """
    method_root=None: 只画 shortest path
    否则：画 shortest(绿) + method(蓝)
    """
    scene_dir_A = os.path.join(ROOT_A, scene_name)
    filesA = sorted_npz_files(scene_dir_A)
    if len(filesA) == 0:
        print(f"[Skip] A missing scene: {scene_name}")
        return False

    d0 = np.load(filesA[0])
    need_keys = ["bounds","map_shape","map","fog_of_war_mask","agent_map_coord","agent_angle","agent_pos"]
    if not all(k in d0 for k in need_keys):
        print(f"[Skip] {scene_name}: missing keys in A")
        return False

    bounds = d0["bounds"].astype(np.float32)
    H, W = map(int, d0["map_shape"])

    # shortest path from A (rc)
    worldA, rcA = load_A_world_and_rc(filesA)
    flip_z = pick_best_flip(worldA, rcA, bounds, H, W)
    path_short = clip_rc(rcA, H, W)

    # official base map
    info0 = npz_to_info_topdown(d0)
    img_rgb = plot_top_down_map(info0, dataset=DATASET_NAME)
    img_bgr = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)

    # draw shortest (green)
    for i in range(1, len(path_short)):
        r1, c1 = path_short[i-1]
        r2, c2 = path_short[i]
        cv2.line(img_bgr, (c1, r1), (c2, r2), COLOR_SHORTEST, 4)

    # draw method (blue) if provided
    if method_root is not None:
        scene_dir_M = os.path.join(method_root, scene_name)
        filesM = sorted_npz_files(scene_dir_M)
        if len(filesM) == 0:
            print(f"[Warn] method missing scene: {method_tag} / {scene_name}")
        else:
            try:
                worldM = load_world_only(filesM)
                rcM = world_to_grid_rc(worldM, bounds, H, W, flip_z=flip_z)
                path_m = clip_rc(rcM, H, W)

                for i in range(1, len(path_m)):
                    r1, c1 = path_m[i-1]
                    r2, c2 = path_m[i]
                    cv2.line(img_bgr, (c1, r1), (c2, r2), COLOR_METHOD, 4)

                # method end marker (optional)
                r_end, c_end = path_m[-1]
                draw_square(img_bgr, (c_end, r_end), size=6, color=COLOR_END, thickness=-1)

            except Exception as e:
                print(f"[Warn] {scene_name} in {method_tag}: {e}")

    # save
    out_path = os.path.join(OUT_DIR, f"{scene_name}__{method_tag}.png")
    cv2.imwrite(out_path, img_bgr)
    return True

# ===== 主循环：只处理 SCENE_LIST =====
saved = 0

for scene_name in SCENE_LIST:
    # 1) shortest-only 图（可选，想要就保留）
    ok = render_one(scene_name, method_root=None, method_tag="shortest")
    saved += int(ok)

    # 2) 每个方法单独一张图：shortest(绿) + method(蓝)
    for method_root in ROOT_OTHERS:
        tag = os.path.basename(method_root).replace("debug_npz_", "")  # e.g. ours/iros/...
        ok = render_one(scene_name, method_root=method_root, method_tag=tag)
        saved += int(ok)

print("Saved images:", saved, "Output dir:", OUT_DIR)


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
2026-01-25 17:25:12.382466: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-25 17:25:12.416811: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-25 17:25:13.12

Saved images: 161 Output dir: /home/Disk/yyz/sound-spaces/vis/topdown_pngs_sep
